In [1]:
import os
import pyaudio
import wave
import soundfile as sf

In [4]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
MAX_RECORD_SECONDS = 12
WAVE_OUTPUT_DIR = "../data/custom/audio/"
TRANSCRIPT_DIR = "../data/custom/transcripts/"
TRANSCRIPT_FILE = "../data/custom/source.txt"

In [3]:
if not os.path.exists(WAVE_OUTPUT_DIR):
    os.makedirs(WAVE_OUTPUT_DIR)

if not os.path.exists(TRANSCRIPT_DIR):
    os.makedirs(TRANSCRIPT_DIR)

In [5]:
def record_audio(filename):
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for _ in range(0, int(RATE / CHUNK * MAX_RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [6]:
def trim_audio(filename):
    data, samplerate = sf.read(filename)
    if len(data) > samplerate * MAX_RECORD_SECONDS:
        data = data[:samplerate * MAX_RECORD_SECONDS]
        sf.write(filename, data, samplerate)

In [7]:
def write_transcript(filename, transcript):
    with open(filename, 'w') as f:
        f.write(transcript)

In [ ]:
def read_transcripts(file):
    with open(file, 'r') as f:
        lines = f.readlines()

    for i, line in enumerate(lines):
        if line.strip() == '':
            continue
        audio_filename = os.path.join(WAVE_OUTPUT_DIR, f'speech_{i+1:02}.wav')
        transcript_filename = os.path.join(TRANSCRIPT_DIR, f'speech_{i+1:02}.txt')

        if os.path.exists(audio_filename) and os.path.exists(transcript_filename):
            continue

        print(f"Please read the following line: {line}")
        input("Press Enter when you're ready to start recording...")

        record_audio(audio_filename)
        trim_audio(audio_filename)
        write_transcript(transcript_filename, line)

        print(f"Recording for line {i+1} saved!\n")
